In [1]:
import pandas as pd
from google.colab import drive
import tensorflow as tf
from pydrive.drive import GoogleDrive
import os
import shutil
drive.mount('/content/gdrive')

!unzip gdrive/My\ Drive/archive.zip > /dev/null
print("Total Images: {}".format(len([name for name in os.listdir('ISIC_2019_Training_Input/ISIC_2019_Training_Input')])))

Mounted at /content/gdrive
Total Images: 25333


In [2]:
def OrganizeImages():
  df_gt = pd.read_csv ('ISIC_2019_Training_GroundTruth.csv')
  df_meta = pd.read_csv ('ISIC_2019_Training_Metadata.csv')

  df_mel = df_gt.loc[(df_gt['MEL'] == 1)]
  df_bcc = df_gt.loc[(df_gt['BCC'] == 1)]
  df_bkl = df_gt.loc[(df_gt['BKL'] == 1)]

  return df_mel, df_bcc, df_bkl

In [7]:
def ExtractImages(df_mel, df_bcc, df_bkl):
  os.mkdir('MEL')
  os.mkdir('MEL/MEL')
  for index, row in df_mel.iterrows():
    image = "ISIC_2019_Training_Input/ISIC_2019_Training_Input/{}.jpg".format(row['image'])
    shutil.move(image, "MEL/MEL")

  os.mkdir('BCC')
  os.mkdir('BCC/BCC')
  for index, row in df_bcc.iterrows():
    image = "ISIC_2019_Training_Input/ISIC_2019_Training_Input/{}.jpg".format(row['image'])
    shutil.move(image, "BCC/BCC")

  os.mkdir('BKL')
  os.mkdir('BKL/BKL')
  for index, row in df_bkl.iterrows():
    image = "ISIC_2019_Training_Input/ISIC_2019_Training_Input/{}.jpg".format(row['image'])
    shutil.move(image, "BKL/BKL")

  print("MEL: {}".format(len([name for name in os.listdir('MEL/MEL')])))
  print("BCC: {}".format(len([name for name in os.listdir('BCC/BCC')])))
  print("BKL: {}".format(len([name for name in os.listdir('BKL/BKL')])))

In [4]:
def SplitData(dir, lesion_type, training, validation, test):
  length = len([name for name in os.listdir(dir)])
  train_size = length * training
  validation_size = length * validation
  test_size = length * test

  try:
    os.mkdir(lesion_type + '/train')
    os.mkdir(lesion_type + '/validation')
    os.mkdir(lesion_type + '/test')
  except OSError as e:
    print("Error: {}.\nContinuing...".format(e.strerror))

  end_index = 0
  names = [name for name in os.listdir(dir)]
  for i in range(int(train_size)):
    image = dir + "/{}".format(names[i])
    shutil.move(image, lesion_type + '/train')
    end_index = i
  
  for i in range(end_index + 1, int(end_index + validation_size)):
    image = dir + "/{}".format(names[i])
    shutil.move(image, lesion_type + '/validation')
    end_index = i

  for i in range(end_index + 1, int(end_index + test_size)):
    image = dir + "/{}".format(names[i])
    shutil.move(image, lesion_type + '/test')

  print_str = lesion_type + ":\n\Train Size: {}\n\tValidation Size: {}\n\tTest Size: {}\n"
  print(print_str.format(len([name for name in os.listdir(lesion_type + '/train')]), len([name for name in os.listdir(lesion_type + '/validation')]), len([name for name in os.listdir(lesion_type + '/test')])))

In [5]:
training = 0.7
validation = 0.15
test = 0.15

df_mel, df_bcc, df_bkl = OrganizeImages()
ExtractImages(df_mel, df_bcc, df_bkl)
SplitData('\nMEL/MEL', 'MEL', training, validation, test)
SplitData('BCC/BCC', 'BCC', training, validation, test)
SplitData('BKL/BKL', 'BKL', training, validation, test)

MEL: 1
BCC: 1
BKL: 1
MEL:
	Test Size: 3165
	Validation Size: 677
	Test Size: 677

BCC:
	Test Size: 2326
	Validation Size: 497
	Test Size: 497

BKL:
	Test Size: 1836
	Validation Size: 392
	Test Size: 392

